In [1]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

device = 'cuda' if torch.cuda.is_available() else 'cpu'\

In [2]:
class trainData(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(12, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

In [30]:
#read in files
file_in = os.getcwd() + "/avacados/avocado.csv"
df = pd.read_csv(file_in)
print(df.head())


   Unnamed: 0        Date  AveragePrice  Total Volume     4046       4225  \
0           0  2015-12-27          1.33      64236.62  1036.74   54454.85   
1           1  2015-12-20          1.35      54876.98   674.28   44638.81   
2           2  2015-12-13          0.93     118220.22   794.70  109149.67   
3           3  2015-12-06          1.08      78992.15  1132.00   71976.41   
4           4  2015-11-29          1.28      51039.60   941.48   43838.39   

     4770  Total Bags  Small Bags  Large Bags  XLarge Bags          type  \
0   48.16     8696.87     8603.62       93.25          0.0  conventional   
1   58.33     9505.56     9408.07       97.49          0.0  conventional   
2  130.50     8145.35     8042.21      103.14          0.0  conventional   
3   72.58     5811.16     5677.40      133.76          0.0  conventional   
4   75.78     6183.95     5986.26      197.69          0.0  conventional   

   year  region  
0  2015  Albany  
1  2015  Albany  
2  2015  Albany  
3  2015 

In [ ]:
#print(df.head)
X = df.iloc[:, 0:-1] #grab first 13 elements for input
y = df.iloc[:, -1]   #seperate last element (target values)

scaler = StandardScaler()
X_train = scaler.fit_transform(X)

In [6]:
#reshape data into a tensor of floats
train_data = trainData(torch.FloatTensor(X_train),torch.FloatTensor(y))
train_loader = DataLoader(dataset=train_data, batch_size=5, shuffle=True)

#build model and pass it to device (CPU or GPU)
model = NeuralNetwork().to(device)
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [25]:
def accuracy(y_pred, y_actual):
    #print(y_pred, y_actual)
    diff = (abs(y_pred - y_actual)/(y_pred + y_actual)).mean()
    diff = (1 - diff) * 100
    #print(diff)
    return diff

In [26]:
epochs = 100
for e in range(epochs):
    epoch_loss = 0
    epoch_acc = 0
    print(f"Epoch {e + 1}\n-------------------------------")
    for features, labels in train_loader:

        features, labels = features.to(device), labels.to(device)
        y_pred = model(features)
        loss = loss_fn(y_pred, labels.unsqueeze(1))
        acc = accuracy(y_pred, labels.unsqueeze(1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    print(f'Epoch {e + 0:03}: | Loss: {epoch_loss / len(train_loader):.5f} | Acc: {acc:.3f}')

print("Done Training!")

Epoch 1
-------------------------------
Epoch 000: | Loss: 3.56090 | Acc: 99.185
Epoch 2
-------------------------------
Epoch 001: | Loss: 3.71723 | Acc: 99.449
Epoch 3
-------------------------------
Epoch 002: | Loss: 3.81223 | Acc: 99.568
Epoch 4
-------------------------------
Epoch 003: | Loss: 3.33987 | Acc: 98.729
Epoch 5
-------------------------------
Epoch 004: | Loss: 3.46746 | Acc: 99.693
Epoch 6
-------------------------------
Epoch 005: | Loss: 3.49470 | Acc: 99.317
Epoch 7
-------------------------------
Epoch 006: | Loss: 3.52524 | Acc: 99.260
Epoch 8
-------------------------------
Epoch 007: | Loss: 3.60855 | Acc: 99.370
Epoch 9
-------------------------------
Epoch 008: | Loss: 3.21427 | Acc: 99.840
Epoch 10
-------------------------------
Epoch 009: | Loss: 3.35218 | Acc: 99.383
Epoch 11
-------------------------------
Epoch 010: | Loss: 3.90428 | Acc: 98.692
Epoch 12
-------------------------------
Epoch 011: | Loss: 3.40690 | Acc: 99.546
Epoch 13
----------------

KeyboardInterrupt: 